<h1>Módulo de Extracción de datos</h1>

Este notebook tiene como finalidad realizar una recabación de twits relacionados al mundial 2022. Para realizar esta extracción de tweets utilizaremos las siguientes herramientas: Tweepy y Pandas.

In [1]:
import tweepy
import time
import pandas as pd
pd.set_option('display.max_colwidth', None)
import re

In [2]:
%run ./twitter_keys.ipynb

In [3]:
# Setup access to API
def connect_to_twitter_OAuth():
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    api = tweepy.API(auth, wait_on_rate_limit=True)  # add proxy='172.30.1.251:6969' if needed
    return api

# Create API object
api = connect_to_twitter_OAuth()

In [4]:
#contenido del tweet
tweets = []
#cantidad de likes del tweet
likes = []
#tiempo en el cual el tweet se creo
time = []
#ID del tweet
tweet_id = []

In [5]:
#add queries to search
queries = ['Mundial','Qatar','#Qatar2022','Messi','Catar']
rtfilter = " -filter:retweets lang:es"
queries = [s + rtfilter for s in queries]

In [6]:
for query in queries:
    for tweet in tweepy.Cursor(api.search_tweets, 
                                     q = query).items(100):
        text = tweet.text.replace('\n',' ')
        clean_tweet = re.sub("@[A-Za-z0-9]+","", text)
        clean_tweet = re.sub("#[A-Za-z0-9]+","", clean_tweet)
        clean_tweet = re.sub(r"http\S+", "", clean_tweet)
        tweets.append(clean_tweet.lower())
        tweet_id.append(tweet.id_str)
        likes.append(tweet.favorite_count)
        time.append(tweet.created_at)

In [9]:
df2 = pd.DataFrame({'tweet_id':tweets_id,'tweets':tweets,'likes':likes,'time':time})

In [10]:
df2.head(500)

,tweet_id,tweets,likes,time
0,1593005176960405504,"hay que ser patriota y cumplir la constitución y también estar orgulloso del primer ministro español,…",0,2022-11-16 22:16:45+00:00
1,1593005173877936129,"la decepción de esta copa mundial será argentina, se tenía que decir… qué pena herirlos.",0,2022-11-16 22:16:44+00:00
2,1593005172871286784,como que se han dado cuenta unos cuantos años tarde de los problemas que iba a tener organizar el mundial en qatar.,0,2022-11-16 22:16:44+00:00
3,1593005172137295875,"no sé si es su perfil pero le ruego deje el sevilla, como socio y accionista ,no ha jugado en el sev…",0,2022-11-16 22:16:44+00:00
4,1593005168143912961,españa se aloja en el mismo lugar que la selección en el mundial de qatar 2022: la selección española ta…,0,2022-11-16 22:16:43+00:00
...,...,...,...,...
495,1593000093078388736,el mediocampista de la selección argentina rodrigo de paul se mostró optimista y remarcó que «llegamos bien» al mun…,0,2022-11-16 21:56:33+00:00
496,1593000026166669312,"vos vas a catar... a catar porongas en tu casa, bobi.",0,2022-11-16 21:56:17+00:00
497,1593000001890058241,o se hacia la foto o perdia el vuelo a catar jajajajja,0,2022-11-16 21:56:11+00:00
498,1592999946437160961,nordeste &gt;&gt;&gt; catar,0,2022-11-16 21:55:58+00:00


In [17]:
from datetime import datetime
now = datetime.now()
df2.to_csv('D:\\FACU\\TF_CSVs\\'+'WorldCupTweets_' + str(now)[:16].replace(':','_') + '.csv',
          index=True,
          encoding='utf-8-sig',
          sep=';'
         )


<h1> Dataset 1: Topic detection and sentiment analysis twitter dataset </h1>

This is a Spanish dataset that is labeled for sentiment analysis and topic detection purposes. For more info: https://github.com/NatashaSvic/NLP_Spanish_Sentiment_Anaylsis_Text_Generation/

In [12]:
import xml.etree.ElementTree as ET

In [13]:
tree = ET.parse('../0_datasets/TASS_data/general_corpus_2012/general-train-tagged-3l.xml')
root = tree.getroot()

In [14]:
train_set = pd.DataFrame({'tweet_id':[],'tweetText':[],'polarity_value':[],'polarity_type':[],'topic':[]})


In [15]:
row=0
for tweet in root:
    tweet_id = 'ID:'+tweet.find('tweetid').text
    #user = tweet.find('user').text
    tweetText = tweet.find('content').text
    lang = tweet.find('lang').text
    polarity_value = tweet.find('sentiments').find('polarity').find('value').text
    polarity_type = tweet.find('sentiments').find('polarity').find('type').text
    topic = tweet.find('topics').find('topic').text
    
    if lang == 'es':
        train_set.loc[row] = [tweet_id,tweetText,polarity_value,polarity_type,topic]
        row+=1

In [16]:
train_set.head(10)

,tweet_id,tweetText,polarity_value,polarity_type,topic
0,ID:142389495503925248,"Salgo de #VeoTV , que día más largoooooo...",NONE,AGREEMENT,otros
1,ID:142389933619945473,@PauladeLasHeras No te libraras de ayudar me/nos. Besos y gracias,NEU,DISAGREEMENT,otros
2,ID:142391947707940864,@marodriguezb Gracias MAR,P,AGREEMENT,otros
3,ID:142416095012339712,"Off pensando en el regalito Sinde, la que se va de la SGAE cuando se van sus corruptos. Intento no sacar conclusiones (lo intento)",N,AGREEMENT,política
4,ID:142422495721562112,Conozco a alguien q es adicto al drama! Ja ja ja te suena d algo!,P,AGREEMENT,otros
5,ID:142424715175280640,"RT @FabHddzC: Si amas a alguien, déjalo libre. Si grita ese hombre es mío era @paurubio...",NONE,AGREEMENT,música
6,ID:142483342040907776,Toca @crackoviadeTV3 . Grabación dl especial Navideño...Mari crismas!,P,AGREEMENT,entretenimiento
7,ID:142493511634259968,Hoy asisitiré en Madrid a un seminario sobre la Estrategia Española de Seguridad organizado por FAES.,NONE,AGREEMENT,política
8,ID:142494476051562496,Buen día todos! Lo primero mandar un abrazo grande a Miguel y a su familia @libertadmontes Hoy podría ser un día para la grandeza humana.,P,AGREEMENT,otros
9,ID:142496796416016384,Desde el escaño. Todo listo para empezar #endiascomohoy en el Congreso http://t.co/Mu2yIgCb,P,AGREEMENT,política
